<a href="https://colab.research.google.com/github/M0hammadTamimi/AI-Model-Training-1/blob/main/finaltrainMT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets openpyxl
!pip install datasets
!pip install -q transformers datasets wandb peft torch pandas openpyxl
# Cell 1: Install Requirements
!pip install -q transformers datasets wandb peft torch pandas openpyxl psutil tkseem

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.9/30.9 MB 61.4 MB/s eta 0:00:00
  

In [1]:
# Step 1: Install Necessary Libraries

# Install PyTorch with GPU support
!pip install torch torchvision torchaudio

# Install Hugging Face Transformers library
!pip install transformers

# Install other required libraries
!pip install datasets wandb pandas openpyxl psutil

# Step 2: Verify CUDA Availability

import torch

# Check if CUDA is available (GPU support)
cuda_available = torch.cuda.is_available()
gpu_name = torch.cuda.get_device_name(0) if cuda_available else "No GPU found"
print(f"CUDA Available: {cuda_available}")
print(f"GPU Name: {gpu_name}")

# Step 3: Check GPU Information

if cuda_available:
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated(0)} bytes")
    print(f"GPU Memory Cached: {torch.cuda.memory_reserved(0)} bytes")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
CUDA Available: False
GPU Name: No GPU found


In [ ]:
# Cell 1: Install Requirements
!pip install -q transformers datasets wandb peft torch pandas openpyxl psutil tkseem

# Cell 2: Import Libraries
import pandas as pd
import wandb
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset
from sklearn.model_selection import train_test_split
from peft import get_peft_model, LoraConfig
import os
from tqdm.auto import tqdm
import subprocess
import psutil

# Cell 3: Mount Drive and Setup
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Initialize wandb
wandb.login()  # You'll need to enter your API key
wandb.init(project="huggingface", entity="mohammadtamimi300-hashmite-tech")

# Cell 4: Display GPU and RAM Information
def display_system_info():
    # GPU Info
    gpu_info = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE)
    print(gpu_info.stdout.decode())

    # RAM Info
    ram_info = psutil.virtual_memory()
    print(f"Total RAM: {ram_info.total / (1024 ** 3):.2f} GB")
    print(f"Available RAM: {ram_info.available / (1024 ** 3):.2f} GB")

display_system_info()

# Cell 5: Load and Prepare Dataset
print("Loading dataset...")
df = pd.read_excel('/content/datasetQA.xlsx')
print(f"Dataset loaded with {len(df)} rows")

# Cell 6: Initialize Tokenizer and Define Preprocessing
# Use AutoTokenizer from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("aubmindlab/bert-base-arabertv2")

def preprocess_function(examples):
    # Convert inputs to strings
    questions = [str(q) if pd.notnull(q) else '' for q in examples["question"]]
    contexts = [str(c) if pd.notnull(c) else '' for c in examples["context"]]
    answers = examples['answer']

    # Tokenize using the Hugging Face tokenizer
    inputs = tokenizer(questions, contexts, truncation=True, padding=True, max_length=512, return_tensors="pt")

    # Calculate start and end positions
    start_positions = []
    end_positions = []

    for i in range(len(answers)):
        answer = answers[i]
        if answer:
            answer_start = contexts[i].find(answer)
            if answer_start == -1:  # Handle case where the answer is not found
                start_positions.append(0)
                end_positions.append(0)
            else:
                start_positions.append(inputs.char_to_token(i, answer_start) or 0)  # Ensure valid token index
                end_position = answer_start + len(answer)
                end_positions.append(inputs.char_to_token(i, end_position - 1) or 0)  # Ensure valid token index
        else:
            start_positions.append(0)
            end_positions.append(0)

    inputs.update({"start_positions": start_positions, "end_positions": end_positions})

    return inputs

# Cell 7: Process Dataset
print("Processing dataset...")
dataset = Dataset.from_pandas(df)
processed_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

# Split dataset into train and validation sets
train_dataset, val_dataset = processed_dataset.train_test_split(test_size=0.1).values()

print(f"Train set: {len(train_dataset)} examples")
print(f"Validation set: {len(val_dataset)} examples")

# Cell 8: Initialize Model and Configure LoRA
print("Initializing model...")
model = AutoModelForQuestionAnswering.from_pretrained(
    "aubmindlab/bert-base-arabertv2",
    return_dict=True
)

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    task_type="QUESTION_ANS",
    target_modules=["query", "key", "value"],
    bias="none",
    modules_to_save=["qa_outputs"]
)

# Apply LoRA
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

# Cell 9: Display Model Info (Trainable Parameters)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f"\nTrainable params: {trainable_params} ||")
print(f"All params: {total_params} ||")
print(f"Trainable%: {trainable_params / total_params * 100:.4f}")

# Cell 10: Configure Training
output_dir = '/content/training_model_output'
logging_dir = '/content/training_model_output/logs'

os.makedirs(output_dir, exist_ok=True)
os.makedirs(logging_dir, exist_ok=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    run_name="lora_qa_full_dataset",  # Unique name for this run
    evaluation_strategy="steps",  # Enable evaluation
    eval_steps=50,  # Evaluate every 50 steps
    logging_steps=50,  # Log every 50 steps
    learning_rate=5e-5,
    per_device_train_batch_size=4,  # Lower batch size for memory efficiency
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4,  # Accumulate gradients to simulate larger batch size
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir=logging_dir,
    report_to="wandb",
    fp16=True,  # Enable mixed-precision training to save memory
    save_strategy="steps",
    save_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    warmup_ratio=0.1,
    push_to_hub=False,
    remove_unused_columns=True,
    group_by_length=True,  # Improves training speed
    dataloader_num_workers=2,  # Parallel data loading
    gradient_checkpointing=True  # Save memory by checkpointing gradients
)

# Use DataCollatorWithPadding to handle padding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Cell 11: Initialize Trainer and Start Training
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)

# Train with proper error handling
print("Starting training...")
try:
    trainer.train()
    print("Training completed successfully!")
except Exception as e:
    print(f"Training error occurred: {str(e)}")
    print("Saving checkpoint model...")
    model.save_pretrained("/content/checkpoint_model")
finally:
    wandb.finish()

# Cell 12: Save Model
print("\nSaving final model...")
try:
    # Save the model
    model.save_pretrained("/content/final_model")
    # Save the tokenizer using the appropriate method
    tokenizer.save_pretrained('/content/final_model')
    print("Model saved successfully!")

    # Create zip file
    !zip -r /content/final_model.zip /content/final_model
    print("Model zipped successfully! ")

    # Save to Drive
    drive_path = "/content/drive/MyDrive/models/final_model.zip"
    os.makedirs(os.path.dirname(drive_path), exist_ok=True)
    !cp /content/final_model.zip "{drive_path}"
    print(f"\nModel saved to Drive at: {drive_path}")

except Exception as e:
    print(f"Error in saving: {str(e)}")

print("Process completed!")


wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Mounted at /content/drive
Fri Dec 27 06:26:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0              29W /  70W |   3531MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------

Map:   0%|          | 0/15323 [00:00<?, ? examples/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train set: 13790 examples
Validation set: 1533 examples
Initializing model...


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


trainable params: 886,274 || all params: 135,490,564 || trainable%: 0.6541

Trainable params: 886274 ||
All params: 135490564 ||
Trainable%: 0.6541
Starting training...


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss
50,4.701900,4.684445
100,4.634200,4.575869
150,4.491800,4.392453
200,4.282900,4.130871
250,3.998400,3.802675
300,3.671700,3.427218
350,3.294500,3.063507
400,2.981100,2.716648
450,2.643700,2.391415
500,2.343200,2.084323


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None


In [18]:
pip install transformers datasets torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

# Load the fine-tuned model and tokenizer from Hugging Face Hub
model_name = "aubmindlab/bert-base-arabertv2"
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Function to answer a question using the fine-tuned model
def answer_question(question, context):
    inputs = tokenizer(question, context, return_tensors="pt", truncation=True, padding=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)

    start_scores = outputs.start_logits
    end_scores = outputs.end_logits

    start_index = torch.argmax(start_scores)
    end_index = torch.argmax(end_scores)

    # Convert token indices back to string and remove special tokens
    answer_tokens = inputs.input_ids[0][start_index:end_index + 1]
    answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

    return answer

# Example usage with your dataset
contexts = [
    "الجامعة الهاشمية تقدم درجات بكالوريوس في مجالات مثل علوم الكمبيوتر والهندسة.",
    "تعليمات منح درجة البكالوريوس في الجامعة الهاشمية تحدد شروط التخرج والمتطلبات.",
    "تم إصدار تعليمات منح درجة البكالوريوس في الجامعة الهاشمية برقم (1084/32/2016) بتاريخ 23/5/2016.",
    "أعلنت الجامعة الهاشمية أن الامتحانات النهائية ستبدأ في بداية يونيو 2024.",
    "الجامعة الهاشمية توفر برامج ماجستير في مجالات متعددة مثل الهندسة والإدارة."
]

questions = [
    "ما هي درجات البكالوريوس التي تقدمها الجامعة الهاشمية؟",
    "ما اسم التعليمات المتعلقة بمنح درجة البكالوريوس في الجامعة الهاشمية؟",
    "ما رقم وتاريخ إصدار تعليمات منح درجة البكالوريوس في الجامعة الهاشمية؟",
    "متى ستبدأ الامتحانات النهائية في الجامعة الهاشمية؟",
    "ما هي برامج الماجستير التي تقدمها الجامعة الهاشمية؟"
]

# Iterate over your dataset and get the answers
for context, question in zip(contexts, questions):
    answer = answer_question(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}")
    print("-" * 50)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv2 and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/611 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/720k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Question: ما هي درجات البكالوريوس التي تقدمها الجامعة الهاشمية؟
Answer: ##عة الهاشمية ؟ الجامعة الهاشمية تقدم درجات بكالوريوس في مجالات مثل علوم الك
--------------------------------------------------
Question: ما اسم التعليمات المتعلقة بمنح درجة البكالوريوس في الجامعة الهاشمية؟
Answer: ##عة الهاشمية ؟ تعليمات منح درجة البكالوريوس في الجامعة الهاشمية تحدد شروط التخرج والمت
--------------------------------------------------
Question: ما رقم وتاريخ إصدار تعليمات منح درجة البكالوريوس في الجامعة الهاشمية؟
Answer: تعليمات منح درجة البكالوريوس في الجامعة الهاشمية ؟ تم إصدار تعليمات منح درجة الب
--------------------------------------------------
Question: متى ستبدأ الامتحانات النهائية في الجامعة الهاشمية؟
Answer: ##عة الهاشمية ؟ أعلنت
--------------------------------------------------
Question: ما هي برامج الماجستير التي تقدمها الجامعة الهاشمية؟
Answer: ##عة الهاشمية ؟ الجامعة الهاشمية توفر برامج ماجستير في مجالات متعددة مثل الهندسة
--------------------------------------------------
